In [42]:
# 시스템
import os
import sys
import re
# 크롤링
import urllib.request
import requests
import json
from scrapy.http import TextResponse
from bs4 import BeautifulSoup
from selenium import webdriver
from tqdm import tqdm

import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [8]:
# 태그 데이터를 삭제하는 코드
def tag_remove(word):

    word = re.sub('(<([^>]+)>)', '', word)

    return word

# 한글만 남기는 코드 -> 태그만 삭제해야함
def preprocess_sentence_kr(w):
    w = w.strip()
    w = re.sub(r"[^0-9가-힣?.!,¿]+", " ", w) # \n도 공백으로 대체해줌
    w = w.strip()
    return w

# 한글 영어만 남기는 코드 -> 태그만 삭제해야함
def preprocess_sentence(w):
    w = w.strip()
    w = re.sub(r"[^0-9가-힣a-zA-Z?.!,¿]+", " ", w) # \n도 공백으로 대체해줌
    w = w.strip()
    return w

In [98]:
def blog_content_parsing(url):
    blog_req = requests.get(url)
    blog_bs = BeautifulSoup(blog_req.text, "html.parser")
    
    result = []
    quote = []
    quote_cnt = 0
    img_cnt = 0
    content, first_img, last_img = "", "" ,""
    try:
        # 현재 여기서 안되는것은 스마트에디터 3로 작성된 경우이다.
        # 이미지 내용 크롤링
        img_craw = blog_bs.select(".se-module-image")
        img_cnt = len(img_craw)

        if img_cnt != 0: # 사진이 있는 경우만 크롤링

            first_img = img_craw[0].select_one(".se-module-image-link > img")["data-lazy-src"]
            last_img = img_craw[img_cnt-1].select_one(".se-module-image-link > img")["data-lazy-src"]
        # 글 내용 크롤링
        for i in blog_bs.select(".se-text"):
            for j in i.select(".se-module-text > p"):
                txt = j.text
                if txt != "\u200b":
                    result.append(txt)
        content = " ".join(result)

        for i in blog_bs.select(".se-quote"):
            quote.append(i.text)
            quote_cnt += 1
    except:
        return False, content,quote, quote_cnt,  first_img, last_img,  img_cnt
        
    return True, content,quote, quote_cnt, first_img, last_img,  img_cnt

In [10]:
# blog가 네이버 블로그인지 아닌지 판별하는 코드

def naver_blog_check(code):
    
    if code[0].isdigit():
        return False
    else:
        return True


# 모든 단어를 하나씩 나누어서 확인하자.
# 

def naver_title_check(search_word, title):
     
    # search_word = preprocess_sentence_kr(search_word)
    
    for words in search_word.split():
        for word in words:
            if word.lower() not in title.lower():
                return True

In [11]:
# 이름이 들어오면 해당 내용을 이용하여 블로그를 파싱한다.

# search_word : 어떤 제품을 검색하는지
# repeat_num : 해당 제품이 제목에서 몇번 안나올때까지 검색을 진행할 지
def item_parsing(search_word, start, display, repeat_num):
   
    blog_search = naver_api(search_word, start, display)
    print("start ++++++++++++++++++++")
    # result : url
    # writer : 작성자 이름
    # code : 블로그 코드
    # title : 블로그 제목
    # description : 블로그 요약 설명
    blog_result = []
    blog_writer = []
    blog_code = []
    blog_title = []
    blog_description = []
    repeat = True
    cnt = 0
    # 제목에서 해당 제품의 이름이 5번 연속 안나온다면 패스
    while repeat:
        
        for blog in blog_search["items"]: 
             # 블로그 코드
            code = blog["link"][blog["link"].find("logNo=") + 6:]
            if naver_blog_check(code): # 네이버 블로그가 아닌경우 다음으로 넘기기
                continue

            # 이름
            writer = blog["bloggerlink"][23:]

            # 제목
            title = tag_remove(blog["title"])
            
            if naver_title_check(search_word, title):
                continue

            # 블로그 요약
            description = tag_remove(blog["description"])
            ## 데이터 추가하는 곳
            # 블로그 코드 추가
            blog_code.append(code)
            # 이름 추가
            blog_writer.append(writer)
            # 제목 추가
            blog_title.append(title)
            # 블로그 요약 추가
            blog_description.append(description)
            # 본문 크롤링을 위한 URL주소 추가
            blog_result.append(f"https://blog.naver.com/PostView.naver?blogId={writer}&logNo={code}")
        if start + display <= blog_search["total"]:
            start += display
            blog_search = naver_api(search_word, start, display)
            print(f"start = {start}")
            print("repeat start ++++++++++++++++++++")
        else:
            print("repeat End")
            repeat = False
    return blog_result, blog_writer, blog_code, blog_title, blog_description

In [12]:
def naver_api(search_word, start, display):
    client_id = "GvNa2sBgFDA6v7ujnaz0"
    client_secret = "Yo0jOskXlZ"
    
    encText = urllib.parse.quote(search_word)
    
    url = "https://openapi.naver.com/v1/search/blog?query=" + encText # json 결과
    url += f"&start={start}"
    url += f"&display={display}"

    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    response = urllib.request.urlopen(request)
    rescode = response.getcode()
    if(rescode==200):
        response_body = response.read()
    else:
        print("Error Code:" + rescode)
    return json.loads(response_body.decode('utf-8'))

In [13]:
def search_word(word):
    result = []
    append_data = ["후기", "리뷰"]
    for data in append_data:
        result.append(word + " " + data)
        
    return result

    

In [ ]:
# columns = ["url", "title", "description", "content", "contnet_len", ]

In [79]:
url = []
sd = ["sdf" ,"sdff"]

In [82]:
url += sd

In [83]:
url

['sdf', 'sdff', 'sdf', 'sdff']

In [101]:
df = pd.DataFrame()
words = search_word("WF-1000XM4")
url, title, description = [], [], []
for word in words:
    blog_result, blog_writer, blog_code, blog_title, blog_description = item_parsing(word, 1, 100, 3)
    url += blog_result
    title += blog_title
    description += blog_description
    
df["url"] = url
df["title"] = title
df["description"] = description

df = df.drop_duplicates(["url"]).reset_index(drop=True)


start ++++++++++++++++++++
start = 101
repeat start ++++++++++++++++++++
start = 201
repeat start ++++++++++++++++++++
start = 301
repeat start ++++++++++++++++++++
repeat End
start ++++++++++++++++++++
start = 101
repeat start ++++++++++++++++++++
start = 201
repeat start ++++++++++++++++++++
start = 301
repeat start ++++++++++++++++++++
start = 401
repeat start ++++++++++++++++++++
repeat End


In [102]:
len(df)

121

In [103]:
df

,url,title,description
0,https://blog.naver.com/PostView.naver?blogId=r...,소니 WF-1000XM4 노이즈 캔슬링 이어폰 개봉 후기...,지난 몇 주간 소니 WF-1000XM4에 대한 사용 후기를 남겼는데 개봉기를 빼놓을...
1,https://blog.naver.com/PostView.naver?blogId=f...,무선 노이즈 캔슬링 이어폰 소니 WF-1000XM4 간단 후기,소니 WF-1000XM4가 발매가 되었습니다. 전작인 WF-1000XM3 발매가 2...
2,https://blog.naver.com/PostView.naver?blogId=s...,준하는 노이즈캔슬링 소니 WF-1000XM4 무선이어폰 후기,기다리고 기다렸던 소니 WF-1000XM4 노이즈캔슬링 이어폰입니다. 이제는 노이즈...
3,https://blog.naver.com/PostView.naver?blogId=l...,소니 무선 노이즈캔슬링 이어폰 WF-1000XM4 내돈내산 리뷰 후기,소니에서 이번에 새로 정식 출시한 WF-1000XM4는 전작 WF-1000XM3에서...
4,https://blog.naver.com/PostView.naver?blogId=x...,"[내돈내산 후기] 음질, 노캔 최강 소니 WF-1000XM4",WF-1000XM4 이렇게 총 다섯 개의 제품들이죠. 이 다섯 개의 제품들을 총 일...
...,...,...,...
116,https://blog.naver.com/PostView.naver?blogId=k...,"소니 WF-1000XM4 한달만에 도착, 간단리뷰",드디어 도착했기에 대충 리뷰를 끄적여보려 한다. 외형 및 구성품 제품 패키지는 환경...
117,https://blog.naver.com/PostView.naver?blogId=l...,소니 WF-1000XM4 리뷰 노이즈 캔슬링 추천 신제품 정보,소니 WF-1000XM4 (1)소니 WF-1000XM4 리뷰 노이즈 캔슬링 추천 신...
118,https://blog.naver.com/PostView.naver?blogId=l...,"소니 WF-1000XM4, 하이엔드 무선 인이어 헤드폰 리뷰","소니 WF-1000XM4 (1)소니 WF-1000XM4, 하이엔드 무선 인이어 헤드..."
119,https://blog.naver.com/PostView.naver?blogId=b...,블루투스 이어폰 끝판왕 SONY WF-1000XM4 + 모든 기능 리뷰,"[네이버 지식백과] LDAC (쇼핑용어사전) 제품 사양 가격은 정가 기준 299,0..."


In [104]:
content_list, quote_list, quote_cnt_list, first_img_list, last_img_list, img_cnt_list = [], [], [], [], [], []
for i in tqdm(range(len(df))):
    _, content, quote, quote_cnt, first_img, last_img, img_cnt =  blog_content_parsing(df.loc[i, "url"])

    content_list.append(content)
    quote_list.append(quote)
    quote_cnt_list.append(quote_cnt)
    first_img_list.append(first_img)
    last_img_list.append(last_img)
    img_cnt_list.append(img_cnt)

df["content"] = content_list
df["quote"] = quote_list
df["quote_cnt"] = quote_cnt_list
df["first_img"] = first_img_list
df["last_img"] = last_img_list
df["img_cnt"] = img_cnt_list


100%|██████████| 121/121 [00:35<00:00,  3.42it/s]


In [105]:
df

,url,title,description,content,quote,quote_cnt,first_img,last_img,img_cnt
0,https://blog.naver.com/PostView.naver?blogId=r...,소니 WF-1000XM4 노이즈 캔슬링 이어폰 개봉 후기...,지난 몇 주간 소니 WF-1000XM4에 대한 사용 후기를 남겼는데 개봉기를 빼놓을...,지난 몇 주간 소니 WF-1000XM4에 대한 사용 후기를 남겼는데 개봉기를 빼놓을...,[],0,https://postfiles.pstatic.net/MjAyMTExMTdfMjcw...,https://postfiles.pstatic.net/MjAyMTExMThfMjY0...,40
1,https://blog.naver.com/PostView.naver?blogId=f...,무선 노이즈 캔슬링 이어폰 소니 WF-1000XM4 간단 후기,소니 WF-1000XM4가 발매가 되었습니다. 전작인 WF-1000XM3 발매가 2...,노이즈 캔슬링 이어폰의 왕의 귀환! 소니 WF-1000XM4가 발매가 되었습니다....,[],0,https://postfiles.pstatic.net/MjAyMTA3MDdfMTYy...,https://postfiles.pstatic.net/MjAyMTA3MDdfMTAw...,18
2,https://blog.naver.com/PostView.naver?blogId=s...,준하는 노이즈캔슬링 소니 WF-1000XM4 무선이어폰 후기,기다리고 기다렸던 소니 WF-1000XM4 노이즈캔슬링 이어폰입니다. 이제는 노이즈...,,[],0,https://postfiles.pstatic.net/MjAyMTA4MjZfNDMg...,,20
3,https://blog.naver.com/PostView.naver?blogId=l...,소니 무선 노이즈캔슬링 이어폰 WF-1000XM4 내돈내산 리뷰 후기,소니에서 이번에 새로 정식 출시한 WF-1000XM4는 전작 WF-1000XM3에서...,소니에서 이번에 새로 정식 출시한 WF-1000XM4는 전작 WF-1000XM3에서...,[],0,https://postfiles.pstatic.net/MjAyMTA2MjlfMjA5...,https://postfiles.pstatic.net/MjAyMTA2MjlfNTQg...,12
4,https://blog.naver.com/PostView.naver?blogId=x...,"[내돈내산 후기] 음질, 노캔 최강 소니 WF-1000XM4",WF-1000XM4 이렇게 총 다섯 개의 제품들이죠. 이 다섯 개의 제품들을 총 일...,"안녕하세요, 밝은불꽃입니다! 요즘 날씨가 더워서 헤드폰 대신 블루투스 이어폰을 들...","[간단한 음질 비교?, 아무튼 결론]",2,https://postfiles.pstatic.net/MjAyMTA2MzBfMjcx...,https://postfiles.pstatic.net/MjAyMTA2MzBfMjYy...,27
...,...,...,...,...,...,...,...,...,...
116,https://blog.naver.com/PostView.naver?blogId=k...,"소니 WF-1000XM4 한달만에 도착, 간단리뷰",드디어 도착했기에 대충 리뷰를 끄적여보려 한다. 외형 및 구성품 제품 패키지는 환경...,기다리고 기다리던 XM4가 출시되었다는 소식을 듣고 부랴부랴 예약을 했던게 벌써 한...,[],0,https://postfiles.pstatic.net/MjAyMTA4MTFfMjIg...,https://postfiles.pstatic.net/MjAyMTA4MTFfNTAg...,35
117,https://blog.naver.com/PostView.naver?blogId=l...,소니 WF-1000XM4 리뷰 노이즈 캔슬링 추천 신제품 정보,소니 WF-1000XM4 (1)소니 WF-1000XM4 리뷰 노이즈 캔슬링 추천 신...,,[],0,,,0
118,https://blog.naver.com/PostView.naver?blogId=l...,"소니 WF-1000XM4, 하이엔드 무선 인이어 헤드폰 리뷰","소니 WF-1000XM4 (1)소니 WF-1000XM4, 하이엔드 무선 인이어 헤드...",,[],0,,,0
119,https://blog.naver.com/PostView.naver?blogId=b...,블루투스 이어폰 끝판왕 SONY WF-1000XM4 + 모든 기능 리뷰,"[네이버 지식백과] LDAC (쇼핑용어사전) 제품 사양 가격은 정가 기준 299,0...",안드로이드폰 최강조합 블루투스 이어폰 최신 V1 프로세서 탑재되어있으며 자그마치 6...,[],0,https://postfiles.pstatic.net/MjAyMTEyMDdfMjM4...,https://postfiles.pstatic.net/MjAyMTEyMDhfNTMg...,30


In [ ]:
def search_word(word):
    result = []
    append_data = ["후기", "리뷰"]
    for data in append_data:
        result.append(word + " " + data)
        
    return result

    

In [64]:
result = blog_content_parsing("https://blog.naver.com/PostView.naver?blogId=skan5422&logNo=222583222026&redirect=Dlog&widgetTypeCall=true&directAccess=false")

In [108]:
df.loc[0, "content"].count("WF")

# for i in df.loc[0, "content"].split():
    

20

In [66]:
tes = ">"
cnt = 0
for i in result[1].split():
    if tes in i:
        cnt += 1

cnt



6

In [59]:
result[1].count("WF-1000XM4")

12

In [63]:
for i in "ㅋㅋㅋ ㅋㅋ ㅋㅋㅋㅋ ㅋ".split():
    print(i.count("ㅋㅋㅋ"))


1
0
1
0


In [ ]:
def keyword_contains(word):
    return 0

def keyword_cnt(word):
    return 0

In [28]:
df = pd.DataFrame()
df["url"] = a
df["writer"] = b
df["title"] = d
df["description"] = e
df["content"] = result


In [29]:
df["협찬"] = 0
df["체험단"] = 0
df["쿠팡"] = 0

df["협찬"][df["content"].str.contains("협찬")] = 1
df["체험단"][df["content"].str.contains("체험단")] = 1
df["쿠팡"][df["content"].str.contains("coupa")] = 1

C:\Users\sungwoo\AppData\Local\Temp\ipykernel_15272\3884111020.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["협찬"][df["content"].str.contains("협찬")] = 1
C:\Users\sungwoo\AppData\Local\Temp\ipykernel_15272\3884111020.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["체험단"][df["content"].str.contains("체험단")] = 1


In [32]:
df[df["content"].str.contains("coupa")]

,url,writer,title,description,content,협찬,체험단


In [74]:
df["coupang"] = 0

In [77]:
df["coupang"][df["content"].str.contains("coupa")] = 1

C:\Users\User\AppData\Local\Temp\ipykernel_10564\285214680.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["coupang"][df["content"].str.contains("coupa")] = 1


In [78]:
df[df["coupang"]==1]

,url,writer,title,description,content,협찬,체험단,coupang


In [105]:
cleanhtml("&nbsp coupa.ng")

'&nbspcoupa.ng'

In [36]:
df.to_csv("소닉_wh-1000xm42.csv", index= False,encoding="utf-8-sig")